In [4]:
import numpy as np
import gpstk
from Toolkit import *
import matplotlib.pyplot as plt
%matplotlib inline

#Receiver IFB 

Podemos Despejar el IFB de receptor con la ecuación

$$vTEC=(TEC_{sl}-b_{s}-b_{r})Cos(\chi)$$
**Conocemos:**<br><br>
$TEC_{sl}$: Tec slant<br>
$\chi$: Zenith, $b_{s}$: Bias Satellite, incluido en rinex y/o ionex<br> 
<br><br>**Incognita:**<br><br>
$b_{r}$: Bias receptor

<img src="Receptor_IFB_Bias.png">

$\sigma_{t,u}$ es la sumatoria de todas las desviaciones en el día con un bías aleatorio. <br>
$\sigma_{u}(n)$ es la desviación de $VTEC$ en un instante $n$ <br>
$VTEC$ Vertical Total Electron Content medido por un solo satélite en tiempo $n$


In [5]:
receiver=["/mnt/zob1324.03.n","/mnt/zob1324.03.o"]

In [6]:
def getIFB_Rec(ofile,nfile): #en cada epoca satelites que ve, L1,L2,P1,P2,Elevacion de cada satelite,IFB satelites (del rinex toca)
    f1,f2=gpstk.L1_FREQ_GPS,gpstk.L2_FREQ_GPS
    info_Rec={}
    
    oheader,odata=gpstk.readRinex3Obs(ofile,strict=True) 
    nheader,ndata=gpstk.readRinex3Nav(nfile)
    
    bcestore = gpstk.GPSEphemerisStore() 
    
    for ndato in ndata:
        ephem = ndato.toGPSEphemeris()
        bcestore.addEphemeris(ephem)
    bcestore.SearchNear()
    
    for observation in odata:
        sats=[satID for satID, datumList in observation.obs.iteritems() if str(satID).split()[0]=="GPS" ] 
        obs_types = np.array([i for i in oheader.R2ObsTypes])
        if 'C1' and 'P2' and 'L1' and 'L2' in obs_types:
            time=observation.time
            for sat in sats:
                eph = bcestore.findEphemeris(sat, time) 
                Tgd=eph.Tgd #esta en metros!
                sat_pos = eph.svXvt(time)
                rec_pos = gpstk.Position(oheader.antennaPosition[0], oheader.antennaPosition[1], oheader.antennaPosition[2]).asECEF()
                elev = oheader.antennaPosition.elvAngle(sat_pos.x)
                azim = oheader.antennaPosition.azAngle(sat_pos.x)
                IPP=rec_pos.getIonosphericPiercePoint(elev, azim, 350000).asECEF()
                t=np.trunc(gpstk.YDSTime(time).sod)
                
                if np.size(np.where(obs_types=='C1'))>0 and np.size(np.where(obs_types=='P2'))>0 and np.size(np.where(obs_types=='L1'))>0 and np.size(np.where(obs_types=='L2'))>0: 
                        
                    C1_idx = np.where(obs_types=='C1')[0][0] 
                    P2_idx = np.where(obs_types=='P2')[0][0]
                    P1=observation.getObs(sat, C1_idx).data 
                    P2=observation.getObs(sat, P2_idx).data

                    L1_idx = np.where(obs_types=='L1')[0][0]
                    L2_idx = np.where(obs_types=='L2')[0][0]
                    L1=observation.getObs(sat, L1_idx).data*f1
                    L2=observation.getObs(sat, L2_idx).data*f2
                    #if P1<5e7 and P2<5e7 and L1<5e7 and L2<5e7: #Distances should be in order of 1e7 meters
                    if t not in info_Rec:
                        info_Rec[t]=[[sat],[L1],[L2],[P1],[P2],[Tgd],[elev]] #agrega nuevo tiempo
                    else:
                        info_Rec[t][0].append(sat) #agrega a uno existente
                        info_Rec[t][1].append(L1)
                        info_Rec[t][2].append(L2)
                        info_Rec[t][3].append(P1)
                        info_Rec[t][4].append(P2)
                        info_Rec[t][5].append(Tgd)
                        info_Rec[t][6].append(elev)

        else:
            print "Needs both L1 and L2 frequencies to compute delay"
    factor_alfa=f2**2/(f1**2-f2**2)
    alfa=1.0/((f1**2/f2**2)-1)
    c=3e8
    h=400e3 #the height of the ionospheric layer, which is assumed to be 400 km in this paper
    Re=6371e3 #Earth Radius
    k=80.62 #(m3/s2). Related to the ionosphere refraction
    
    bias=np.arange(-30,30,0.001)
    bias*=10e-9
    vec_sum_desv=[] #vector with all sumdesv for every bias
    for bi in bias: #Cada candid
        sumdesv=0 #sum of all standar desv  of that they with bias "bi"
        for t in info_Rec:
            Mt=len(info_Rec[t][0]) #number of satellites observed at time t
            TEC=[]
            for m in range(Mt):
                L1,L2=info_Rec[t][1][m],info_Rec[t][2][m]
                P1,P2=info_Rec[t][3][m],info_Rec[t][4][m]
                tgd,elev=info_Rec[t][5][m]/3e8,info_Rec[t][6][m]
                #**********TEC Slanth Path ********************
                #Computed with carrier-phase
                TECsll=2*(f1*f2)**2*(L1-L2)/(k*(f1**2-f2**2))
                #Computed with Code pseudorange
                TECslp=2*(f1*f2)**2*(P1-P2)/(k*(f1**2-f2**2)) 
                #"Baseline" observations with elevation angle < 10 degrees may be affected by multipath 
                Brs=((TECslp-TECsll)*np.sin(elev)**2)/np.sin(elev)**2
                TECsl=TECsll+Brs
                #*********************************************
                zenith=np.arcsin((Re*np.cos(elev)/(Re+h)))
                vTEC=(TECsl-tgd-bias)*np.cos(zenith)
                TEC.append(vTEC)
            desv_u=np.std(TEC) #sigma_u en tiempo n=t
            sumdesv+=desv_u
        vec_sum_desv.append(sumdesv)
    best_bias=bias[np.argmin(vec_sum_desv)]
    return best_bias
        
            


In [7]:
bias_rec=getIFB_Rec("/mnt/zob1324.03.o","/mnt/zob1324.03.n")

KeyboardInterrupt: 